In [3]:
import numpy as np
import yroot
import yleaf
from ytool import *

In [ ]:
def reducebackup(Tree:Treebase, iout:int, resultdir=None):
    prefix = f"[Reduce Backup file] ({iout})"
    clock = timer(text=prefix, verbose=Tree.verbose, debugger=Tree.debugger)

    if not os.path.isfile(f"{resultdir}ytree_{iout:05d}_temp.pickle"):
        raise FileNotFoundError(f"`{resultdir}ytree_{iout:05d}_temp.pickle` is not found!")
    file, _ = pklload(f"{resultdir}ytree_{iout:05d}_temp.pickle")
    if isinstance(file, dict):
        keys = list(file.keys())
        count = 0
        for key in keys:
            gals = file[key]['gal'] if count==0 else np.hstack((gals, file[key]['gal']))
            count += 1
        pklsave(gals, f"{resultdir}ytree_{iout:05d}.pickle", overwrite=True)
        dprint_(f"{prefix} Save `{resultdir}ytree_{iout:05d}.pickle`", Tree.debugger)
        os.remove(f"{resultdir}ytree_{iout:05d}_temp.pickle")
        dprint_(f"{prefix} Remove `{resultdir}ytree_{iout:05d}_temp.pickle`", Tree.debugger)
    clock.done()

In [11]:
data = pklload("/storage3/Clusters/24954/YoungTree/ytree_00157_temp.pickle")

In [12]:
data[0].keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

In [13]:
data[0][3]

{'gal': (95457, 3, 157, 0.85, 4.5792133e+10, 0.22736067, 0.7740321, 0.42071304, -106.64914, -574.0463, -213.47754, 0.00025963, 0.00021268, 4.5792133e+10, array([[156,   4],
        [155,   2],
        [154,   5],
        [153,   9],
        [152,  11]]), array([[3.69412913, 0.99079718, 0.99063713, 0.712906  , 0.99978882],
        [3.09831748, 0.98766664, 0.98775802, 0.12300279, 0.99989003],
        [3.30502135, 0.98465126, 0.98515768, 0.33561743, 0.99959499],
        [3.19321849, 0.98314017, 0.98302193, 0.22736019, 0.9996962 ],
        [3.20353565, 0.98356024, 0.98305672, 0.2370903 , 0.9998284 ]]), array([[158,   5],
        [159,   6],
        [160,   7],
        [161,   2],
        [162,   9]]), array([[3.25738858, 0.99065359, 0.99039579, 0.27643164, 0.99990755],
        [3.28121333, 0.98850323, 0.98777995, 0.30554562, 0.99938452],
        [3.59746998, 0.98649655, 0.98488287, 0.62703542, 0.99905515],
        [3.15327197, 0.98419779, 0.98328499, 0.18603559, 0.99975359],
        [3.312

In [3]:
isind = large_isind

In [4]:
isind

CPUDispatcher(<function large_isind at 0x7fd436cb4ca0>)

In [7]:
from numba.core.config import NUMBA_NUM_THREADS
NUMBA_NUM_THREADS

80

In [2]:
import numpy as np
a = np.array([1])
np.percentile(a, q=[16,50,84])

array([1., 1., 1.])

In [73]:
def a1(a,b):
    return np.sum(a*b)

def a2(a,b):
    return np.convolve( a,b[::-1], mode='valid' )[0]

import numba as nb
@nb.njit(fastmath=True)
def a3(a:np.ndarray,b:np.ndarray)->float:
    n:int = len(a)
    s:float = 0.
    for i in nb.prange(n):
        s += a[i]*b[i]
    return s

@nb.jit(parallel=True)
def a4(a,b):
    n = len(a)
    result = np.full(n, 0., dtype=np.float64)
    for i in nb.prange(n):
        result[i] = a[i]*b[i]
    return np.sum(result)

In [35]:
a = np.random.rand(100000)
b = np.random.rand(100000)

In [72]:
a1(a,b), a2(a,b), a3(a,b)

(24919.238693341762, 24919.23869334176, 24919.238693341747)

In [49]:
%timeit -n 10000 -r 3 a1(a,b)

119 µs ± 435 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [47]:
%timeit -n 10000 -r 3 a2(a,b)

17.9 µs ± 196 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [76]:
%timeit -n 10000 -r 3 a3(a,b)

53.9 µs ± 130 ns per loop (mean ± std. dev. of 3 runs, 10,000 loops each)


In [10]:
np.convolve( a,b[::-1], mode='valid' )[0]

18

In [9]:
import psutil
import random
import threading
import time

class DisplayCPU(threading.Thread):
    def __enter__(self):
        print("enter")
        self.start()
        self.ref = time.time()
        print("enter end")

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("exit")
        self.stop()
        print("exit end")

    def run(self):
        print("run")
        self.go = False
        self.running = True

        currentProcess = psutil.Process()

        while self.running and (time.time()-self.ref < 10):
            print(currentProcess.cpu_percent(interval=1))
        print("run end")

    def stop(self):
        print("stop")
        self.ref = -1
        self.running = False
        print("stop end")

# ----

def i_hate_this():
    print("hate")
    tab = []
    for i in range(1000000):
        tab.append(random.randint(1, 10000))
    tab.sort()
    print("hate end")
    return tab

In [11]:
i_hate_this.__name__

'i_hate_this'

In [16]:
import numpy as np
isind = lambda a,b: np.where(np.isin(a,b))[0]

In [18]:
isin = np.isin
isin(np.arange(10), np.array([1,2,3]))

array([False,  True,  True,  True, False, False, False, False, False,
       False])

In [17]:
isind(np.arange(10), np.array([1,2,3]))

array([1, 2, 3])

In [26]:
display_cpu = DisplayCPU()

display_cpu.start()
try:
    result = i_hate_this()
finally: # stop thread even when I press Ctrl+C
    display_cpu.stop()

97.9
84.0
78.5
84.0
112.2
115.3
86.9
98.0
98.2
83.5
98.2
118.9


In [10]:
display_cpu = DisplayCPU()
display_cpu.ref = time.time()
with display_cpu:
    i_hate_this()

enter
run
enter end
hate
hate end
exit
stop
stop end
exit end


70.1
run end


In [4]:
from rur import uri
from rur.sci import photometry

In [5]:
backups = pklload("/storage3/Clusters/07206/YoungTree/ytree_00117.pickle")

In [7]:
backups.shape

(392,)

In [9]:
backups[0], backups[0].dtype

((135956, 1, 117, 0.65000004, 6.341491e+10, 0.22753587, 0.38074753, 0.5141741, 197.13655, 215.45401, 54.09123, 0.000851, 0.00040146, 6.051862e+10, array([[116,   1],
        [115,   1],
        [114,   1],
        [113,   3],
        [113, 366],
        [112,   2],
        [112, 360]]), array([[3.77491877, 0.95820116, 0.94262481, 0.8853901 , 0.98870271],
        [3.67953292, 0.94513646, 0.91963217, 0.8340207 , 0.98074359],
        [3.52424307, 0.94355837, 0.89675284, 0.71883164, 0.96510023],
        [3.18660495, 0.8226217 , 0.87902973, 0.49747592, 0.9874776 ],
        [1.73318215, 0.10988932, 0.96580602, 0.23057275, 0.42691407],
        [2.83970921, 0.73514534, 0.87760714, 0.28467841, 0.94227833],
        [1.89516662, 0.18489239, 0.93958878, 0.22794081, 0.54274464]]), array([[118,   1],
        [119,   1],
        [120,   1],
        [120, 323],
        [121,   1],
        [121, 320],
        [122,   1],
        [122, 318]]), array([[3.80362497, 0.94360824, 0.95787307, 0.91105617, 0.99

In [10]:
backup = pklload("/storage3/Clusters/07206/YoungTree/ytree_00010_temp.pickle")

In [27]:
import sys
sys.getsizeof(backup[0][1]['saved']['matchrate'][15][1][1])

112

In [29]:
backup[0][1]['saved']['matchrate'][15]

{1: (0.8966794973880926,
  array([ True,  True,  True,  True, False,  True,  True,  True, False,
          True,  True,  True,  True,  True,  True,  True, False,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False,  True,  True,  True,  True, False,  True, False,
          True,  True,  True,  True,  True,  True, False,  True,  True,
          True,  True,  True,  True,  True,  True,  True, False, False,
          True,  True,  True,  True,  True,  True, False, False, False,
          True,  True,  True,  True,  True,  True, False,  True,  True,
         False, False, False,  True,  True, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False,  True,  True, False,  True, False,  True,  True,
          True,  True,  True,  True, False, False,  True,  True, False,
          True,  True,  True, False,  True,  True, False,  True,  True,
          True,  True,  True,  True,  T